In [3]:
import pandas as pd
import numpy as np
import pyodbc


In [227]:
df_apartment = pd.read_excel(r"D:\Jupyter_Notebooks\Apartment_Prices_In_Egypt\Web Scraping Scripts\propertyFinder_apartments.xlsx")
df_villa = pd.read_excel(r"D:\Jupyter_Notebooks\Apartment_Prices_In_Egypt\Web Scraping Scripts\propertyFinder_villa.xlsx")
df_chalet = pd.read_excel(r"D:\Jupyter_Notebooks\Apartment_Prices_In_Egypt\Web Scraping Scripts\propertyFinder_chalet.xlsx")

In [228]:
df = pd.concat([df_apartment, df_villa, df_chalet], ignore_index=True)
df.head()

,Type,Location,Area,Number Of Bedrooms,Number Of Bathrooms,Price
0,Apartment,"Hyde Park, New Cairo City, Cairo",164 sqm,3,3.0,"8,400,000 EGP"
1,Apartment,"The Waterway View, 90 Street, The 5th Settleme...",132 sqm,2,2.0,"8,257,000 EGP"
2,Apartment,"Mountain View Hyde Park, 5th Settlement Compou...",202 sqm,3,4.0,"9,500,000 EGP"
3,Apartment,"The Icon Residence, 5th Settlement Compounds, ...",158 sqm,3,3.0,"6,400,000 EGP"
4,Apartment,"Mountain View iCity, 5th Settlement Compounds,...",170 sqm,3,3.0,"5,750,000 EGP"


In [229]:
def split_location(location):
    parts = [p.strip() for p in location.split(",")]
    governorate = parts[-1]
    city = parts[-2]
    neighborhood = ', '.join(parts[:-2]) if len(parts) > 2 else None
    return pd.Series([neighborhood, city, governorate])


In [230]:
df[['Neighborhood', 'City', 'Governorate']] = df['Location'].apply(split_location)
df = df.drop(columns=['Location'], axis=1)
df.head()

,Type,Area,Number Of Bedrooms,Number Of Bathrooms,Price,Neighborhood,City,Governorate
0,Apartment,164 sqm,3,3.0,"8,400,000 EGP",Hyde Park,New Cairo City,Cairo
1,Apartment,132 sqm,2,2.0,"8,257,000 EGP","The Waterway View, 90 Street, The 5th Settlement",New Cairo City,Cairo
2,Apartment,202 sqm,3,4.0,"9,500,000 EGP","Mountain View Hyde Park, 5th Settlement Compou...",New Cairo City,Cairo
3,Apartment,158 sqm,3,3.0,"6,400,000 EGP","The Icon Residence, 5th Settlement Compounds, ...",New Cairo City,Cairo
4,Apartment,170 sqm,3,3.0,"5,750,000 EGP","Mountain View iCity, 5th Settlement Compounds,...",New Cairo City,Cairo


In [231]:
map = {
    "Area" : "Area SQM"
}
df = df.rename(columns= map)
df.head()

,Type,Area SQM,Number Of Bedrooms,Number Of Bathrooms,Price,Neighborhood,City,Governorate
0,Apartment,164 sqm,3,3.0,"8,400,000 EGP",Hyde Park,New Cairo City,Cairo
1,Apartment,132 sqm,2,2.0,"8,257,000 EGP","The Waterway View, 90 Street, The 5th Settlement",New Cairo City,Cairo
2,Apartment,202 sqm,3,4.0,"9,500,000 EGP","Mountain View Hyde Park, 5th Settlement Compou...",New Cairo City,Cairo
3,Apartment,158 sqm,3,3.0,"6,400,000 EGP","The Icon Residence, 5th Settlement Compounds, ...",New Cairo City,Cairo
4,Apartment,170 sqm,3,3.0,"5,750,000 EGP","Mountain View iCity, 5th Settlement Compounds,...",New Cairo City,Cairo


In [232]:
df['Area SQM'] = df['Area SQM'].str.replace("sqm", "", regex=False).str.split()
df.head()

,Type,Area SQM,Number Of Bedrooms,Number Of Bathrooms,Price,Neighborhood,City,Governorate
0,Apartment,[164],3,3.0,"8,400,000 EGP",Hyde Park,New Cairo City,Cairo
1,Apartment,[132],2,2.0,"8,257,000 EGP","The Waterway View, 90 Street, The 5th Settlement",New Cairo City,Cairo
2,Apartment,[202],3,4.0,"9,500,000 EGP","Mountain View Hyde Park, 5th Settlement Compou...",New Cairo City,Cairo
3,Apartment,[158],3,3.0,"6,400,000 EGP","The Icon Residence, 5th Settlement Compounds, ...",New Cairo City,Cairo
4,Apartment,[170],3,3.0,"5,750,000 EGP","Mountain View iCity, 5th Settlement Compounds,...",New Cairo City,Cairo


In [233]:
def extract_area(x):
    if isinstance(x, list) and len(x) > 0:
        return x[0]
    else:
        return None

df['Area SQM'] = df['Area SQM'].apply(extract_area)
df.head()

,Type,Area SQM,Number Of Bedrooms,Number Of Bathrooms,Price,Neighborhood,City,Governorate
0,Apartment,164,3,3.0,"8,400,000 EGP",Hyde Park,New Cairo City,Cairo
1,Apartment,132,2,2.0,"8,257,000 EGP","The Waterway View, 90 Street, The 5th Settlement",New Cairo City,Cairo
2,Apartment,202,3,4.0,"9,500,000 EGP","Mountain View Hyde Park, 5th Settlement Compou...",New Cairo City,Cairo
3,Apartment,158,3,3.0,"6,400,000 EGP","The Icon Residence, 5th Settlement Compounds, ...",New Cairo City,Cairo
4,Apartment,170,3,3.0,"5,750,000 EGP","Mountain View iCity, 5th Settlement Compounds,...",New Cairo City,Cairo


In [234]:
print(df.loc[df['Area SQM'].isna(), ['Area SQM']])

      Area SQM
4814      None
40215     None
40452     None
41193     None


In [235]:
df = df.drop(labels=[4814, 40215, 40452, 41193])
print(df.loc[df['Area SQM'].isna(), ['Area SQM']])

Empty DataFrame
Columns: [Area SQM]
Index: []


In [236]:
cols = ['Area SQM', 'Number Of Bedrooms', 'Number Of Bathrooms']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df.head()

,Type,Area SQM,Number Of Bedrooms,Number Of Bathrooms,Price,Neighborhood,City,Governorate
0,Apartment,164.0,3.0,3.0,"8,400,000 EGP",Hyde Park,New Cairo City,Cairo
1,Apartment,132.0,2.0,2.0,"8,257,000 EGP","The Waterway View, 90 Street, The 5th Settlement",New Cairo City,Cairo
2,Apartment,202.0,3.0,4.0,"9,500,000 EGP","Mountain View Hyde Park, 5th Settlement Compou...",New Cairo City,Cairo
3,Apartment,158.0,3.0,3.0,"6,400,000 EGP","The Icon Residence, 5th Settlement Compounds, ...",New Cairo City,Cairo
4,Apartment,170.0,3.0,3.0,"5,750,000 EGP","Mountain View iCity, 5th Settlement Compounds,...",New Cairo City,Cairo


In [237]:
df[cols] = df[cols].astype('Int64')
df.dtypes

Type                   object
Area SQM                Int64
Number Of Bedrooms      Int64
Number Of Bathrooms     Int64
Price                  object
Neighborhood           object
City                   object
Governorate            object
dtype: object

In [238]:
df['Price'] = df['Price'].str.replace("EGP", "", regex=False).str.strip()
df['Price']


0         8,400,000
1         8,257,000
2         9,500,000
3         6,400,000
4         5,750,000
            ...    
47291    13,795,000
47292    12,600,000
47293     9,910,988
47294    20,740,000
47295    11,500,000
Name: Price, Length: 47292, dtype: object

In [239]:
df['Price'] = df['Price'].str.replace(',', '', regex=False)
df['Price'] = df['Price'].astype('float64')
df['Price']

0         8400000.0
1         8257000.0
2         9500000.0
3         6400000.0
4         5750000.0
            ...    
47291    13795000.0
47292    12600000.0
47293     9910988.0
47294    20740000.0
47295    11500000.0
Name: Price, Length: 47292, dtype: float64

In [240]:
na_counts = df.isna().sum()
print(na_counts)

Type                      0
Area SQM                630
Number Of Bedrooms     1393
Number Of Bathrooms     606
Price                     0
Neighborhood            292
City                      0
Governorate               0
dtype: int64


In [241]:
df = df.dropna(subset=['Area SQM'])
df['Number Of Bedrooms'].fillna(0, inplace=True)
df['Number Of Bathrooms'].fillna(0, inplace=True)
df['Neighborhood'].fillna("None", inplace=True)
print(df.isna().sum())

Type                   0
Area SQM               0
Number Of Bedrooms     0
Number Of Bathrooms    0
Price                  0
Neighborhood           0
City                   0
Governorate            0
dtype: int64


C:\Users\DELL\AppData\Local\Temp\ipykernel_11468\3409515152.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Number Of Bedrooms'].fillna(0, inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_11468\3409515152.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [242]:
df.to_excel("Houses_Table.xlsx")

In [246]:
# 2. SQL Server connection
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=AMAZIN;"
    "DATABASE=Egypt_Houses_Prices;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

# 3. Define table name
table = "HousesTable"

# Build CREATE TABLE columns

cursor.execute(f"DROP TABLE IF EXISTS {table}")
conn.commit()

columns_sql = []
for col in df.columns:
    if df[col].dtype in ['Int64', 'int64']:
        columns_sql.append(f"[{col}] INT")
    elif df[col].dtype in ['float64', 'Float64']:
        columns_sql.append(f"[{col}] DECIMAL(18,2)")
    else:
        columns_sql.append(f"[{col}] NVARCHAR(MAX)")

create_table_query = f"CREATE TABLE {table} ({', '.join(columns_sql)})"
cursor.execute(create_table_query)
conn.commit()

# 5. Insert rows
insert_query = f"INSERT INTO {table} VALUES ({', '.join('?' * len(df.columns))})"

for _, row in df.iterrows():
    values = [None if pd.isna(v) else v for v in row]
    cursor.execute(insert_query, values)

conn.commit()
conn.close()

print("Data imported successfully!")

Data imported successfully!
